In [1]:
import numpy as np
import random
import itertools
import pandas as pd
import multiprocessing, threading
import math
import scipy.stats as ss
import time
import pickle
import re
import copy

from opyenxes.model.XLog import XLog
from opyenxes.data_in.XUniversalParser import XUniversalParser
from opyenxes.classification.XEventAttributeClassifier import XEventAttributeClassifier

from prefixspan import PrefixSpan

from tqdm import *

In [2]:
### configuration
data_path = "../logs/bpic2011.xes"
traces_picklepath = data_path.replace(".xes", "_raw_traces.pickled")
traces_tmppath = data_path.replace(".xes", "_traces_tmp.pickled")
traces_finalpath = data_path.replace(".xes", "_traces_encoded.pickled")
traces_dictionarypath = data_path.replace(".xes", "_dictionaries.pickled")
target_column = "concept:name"
categorical_feature_names = ["concept:name", "Specialism code", "org:group"]
### configuration end

In [ ]:
with open(data_path) as bpic_file:
    eventlog = XUniversalParser().parse(bpic_file)[0]

In [ ]:
ncores = multiprocessing.cpu_count()
ntraces = len(eventlog)

## Extract data trace-wise from XES format and enrich with BOS/EOS markers

In [ ]:
# collect all attributes
column_names = []

for event in eventlog[0]:
    for attribute in event.get_attributes():
        column_names.append(attribute)
        
column_names = set(column_names) # remove duplicates
column_names = list(column_names)

def create_dataframe_from_trace(t):
    df = pd.DataFrame(columns=column_names, index=range(0,len(t)))
    for event_idx, event in enumerate(t):
        event_attributes = event.get_attributes()
        df.iloc[event_idx]["__case_id"] = 0
        
        for attribute in event_attributes:
            df[attribute].values[event_idx] = event_attributes[attribute].get_value()
    
    return df

ppool = multiprocessing.Pool(ncores)
traces = []
with tqdm(total=len(eventlog), desc="Converting XES traces to Pandas dataframes", unit="traces") as pbar:
    for i, _ in tqdm(enumerate(ppool.imap(create_dataframe_from_trace, eventlog))):
        pbar.update()
        traces.append(_)
        
del eventlog

In [ ]:
pickle.dump(traces, open(traces_picklepath, "wb"))

In [30]:
traces = pickle.load(open(traces_picklepath, "rb"))

## Eliminate correlated or unimportant features

In [ ]:
def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
        https://stackoverflow.com/questions/46498455/categorical-features-correlation"""
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

for col_a,col_b in itertools.product(eventlog_df.columns, repeat=2): 
    candidate = pd.crosstab(eventlog_df[col_a], eventlog_df[col_b]).as_matrix()
    print("{: >30} {: >30} {: >20}".format(col_a, col_b, cramers_v(candidate)))

In [31]:
# lifecyle:transition is always "complete"
# Producer code correlates perfectly with org:group
# Activity code correlates perfectly with concept:name
for t in traces:
    t.drop(columns=["lifecycle:transition", "Producer code", "Activity code", "Section"], inplace=True)

## Create standard feature set

In [32]:
# Set data types for columns that were not autodetected
for i in range(0,len(traces)):
    traces[i]["Specialism code"] = pd.to_numeric(traces[i]["Specialism code"], errors="ignore")
    traces[i]["Number of executions"] = pd.to_numeric(traces[i]["Number of executions"], errors="ignore")

eventlog_df = pd.concat(traces, ignore_index=True)

## Create SP2 feature set

In [ ]:
# https://stackoverflow.com/questions/42636765/how-to-set-all-the-values-of-an-existing-pandas-dataframe-to-zero
# This one-hot encodes all entries in concept:name column for later incrementation once it has been seen
# sp2_features = pd.get_dummies(eventlog_df["concept:name"], prefix="SP2") # can't use windowed representation here as it might skew distribution of values
# eventlog_sp2_df = process_results.copy(deep=True)
# sp2_features    = sp2_features.drop(sp2_features.index[sp2_features.index[len(eventlog_sp2_df):]])
# assert(len(sp2_features) == len(eventlog_sp2_df))

In [33]:
# loop through every trace and encode the presence of an activity
sp2_prefix = "SP2_"
activity_labels = [ "{0}{1}".format(sp2_prefix,a) for a in eventlog_df["concept:name"].unique() ]

def enrich_trace_with_sp2(t):
    sp2_df = pd.DataFrame(columns=activity_labels, index=range(0,len(t)), dtype=np.bool)
    for col in sp2_df.columns: sp2_df[col].values[:] = 0
    sp2_df["{0}{1}".format(sp2_prefix, t["concept:name"][0])].values[0]  = 1
    
    for i in range(1,len(t)):
        first_activity_name = t["concept:name"].iloc[i]
        col = "{0}{1}".format(sp2_prefix,first_activity_name)
        
        sp2_df.values[i] = sp2_df.values[i-1]
        sp2_df[col].values[i] = 1
        
    return sp2_df

ppool = multiprocessing.Pool(ncores)
sp2_traces = []
with tqdm(total=len(traces), desc="Enriching traces with SP2 features", unit="traces") as pbar:
    for i, _ in tqdm(enumerate(ppool.imap(enrich_trace_with_sp2, traces))):
        pbar.update()
        sp2_traces.append(_)

Enriching traces with SP2 features:  50%|████▉     | 570/1143 [00:02<00:02, 234.99traces/s]
570it [00:02, 233.79it/s]
Enriching traces with SP2 features:  70%|██████▉   | 797/1143 [00:03<00:01, 259.22traces/s]
801it [00:03, 233.34it/s]
Enriching traces with SP2 features:  82%|████████▏ | 933/1143 [00:04<00:00, 234.09traces/s]
933it [00:04, 236.61it/s]
Enriching traces with SP2 features: 100%|██████████| 1143/1143 [00:04<00:00, 231.24traces/s]


## Enrich with PrefixSpan features

In [34]:
def print_patterns(pt):
    for p in pt:
        print("Support: {0}%".format(100*p[0]/len(traces)))
        for n in p[1]:
            print("    > ", int_to_event[n])
        print()

# since most patterns begin and end with the <eos> and <bos> markers, the features only become valuable towards the end...
events       = eventlog_df["concept:name"].unique()
event_to_int = dict((c, i) for i,c in enumerate(events) if c not in ["<bos>","<eos>"])
int_to_event = dict((i, c) for i,c in enumerate(events) if c not in ["<bos>","<eos>"])

In [35]:
# Prefixspan requires an array of arrays with one subarray for every trace
encoded_traces = [ t["concept:name"].map(event_to_int).tolist() for t in traces ]
prefixspan_traces = PrefixSpan(encoded_traces)
closed_sequences = prefixspan_traces.topk(25, closed=True) # support is how often the subsequence appears in total
# http://sequenceanalysis.github.io/slides/analyzing_sequential_user_behavior_part2.pdf, slide 5
# print_patterns(ps_topkc)

# only take subsequence which are at a certain level of support? like if ss[0]/len(traces) < .90
#ps_topkc = list(filter(lambda x: x[0]/len(traces) > .90, ps_topkc))
closed_sequences = [ p[1] for p in closed_sequences ]
pftrace_args = [ (t, closed_sequences[:], event_to_int) for t in traces ] # enrich traces with copy of mined subsequences

In [36]:
def wrapped__enrich_trace_with_subseq(args):
    return enrich_trace_with_subseq(*args)

def enrich_trace_with_subseq(t, ps, event_to_int):
    col_prefix = "PFS_"
    subseq_labels = [ "{0}{1}".format(col_prefix,ss_idx) for ss_idx, ss in enumerate(ps) ]
    subseq_df = pd.DataFrame(columns=subseq_labels, index=range(0,len(t)), dtype=np.bool)
    
    for col in subseq_df.columns: subseq_df[col].values[:] = 0
    for i in range(0,len(t)): # loop through sequence, prune items from mined sequences, and once a subsequence array is empty, this subsequence has occured :)
        activity_code = event_to_int.get(t["concept:name"].iloc[i], None)
        
        for subseq_idx in range(0,len(ps)):
            if ps[subseq_idx] == []:
                continue
            if ps[subseq_idx][0] == activity_code:
                ps[subseq_idx].pop(0)
                if ps[subseq_idx] == []:
                    subseq_df.values[i:,subseq_idx] = 1
        
    return subseq_df

ppool = multiprocessing.Pool(ncores)
pf_traces = []

with tqdm(total=len(pftrace_args), desc="Enriching traces with mined subsequence features", unit="traces") as pbar:
    for i, _ in tqdm(enumerate(ppool.imap(wrapped__enrich_trace_with_subseq, pftrace_args))):
        pbar.update()
        pf_traces.append(_)

Enriching traces with mined subsequence features:   0%|          | 0/1143 [00:00<?, ?traces/s]
Enriching traces with mined subsequence features:   1%|          | 13/1143 [00:00<00:13, 85.92traces/s]
Enriching traces with mined subsequence features:   5%|▍         | 55/1143 [00:00<00:09, 112.50traces/s]
Enriching traces with mined subsequence features:   7%|▋         | 78/1143 [00:00<00:08, 129.00traces/s]
Enriching traces with mined subsequence features:  10%|█         | 117/1143 [00:00<00:06, 155.07traces/s]
Enriching traces with mined subsequence features:  16%|█▌        | 182/1143 [00:00<00:04, 200.68traces/s]
Enriching traces with mined subsequence features:  19%|█▉        | 219/1143 [00:00<00:04, 230.16traces/s]
Enriching traces with mined subsequence features:  23%|██▎       | 266/1143 [00:00<00:03, 271.15traces/s]
Enriching traces with mined subsequence features:  28%|██▊       | 318/1143 [00:00<00:02, 308.59traces/s]
Enriching traces with mined subsequence features:  31%|███▏  

## Dictionary encoding for categorical features

In [37]:
# Convert timestamp to total running time
bos_idx = 0
for i in range(0, len(traces)):
    tlen = len(traces[i])-1
    dfs = traces[i]["time:timestamp"] - traces[i]["time:timestamp"][bos_idx]
    traces[i]["time:timestamp"] = dfs.map(lambda d: int(d.total_seconds()/(60*60))) # convert to hours
    
# Create dictionaries here
feature_dict = {}
for cf in categorical_feature_names:
    cf_dict = { 'to_int': {}, 'to_cat': {} }
    events = eventlog_df[cf].unique().tolist()
    if cf == target_column: events.append("<EOS>")
    cf_dict['to_int'] = dict((c, i) for i, c in enumerate(events))
    cf_dict['to_cat'] = dict((i, c) for i, c in enumerate(events))
    feature_dict[cf] = cf_dict
    
# Concatenate all features into one feature dataframe per trace
for i in range(0,len(traces)):
    targets = traces[i][target_column].shift(-1).to_frame("TARGET")
    targets.values[len(traces[i])-1] = "<EOS>"
    targets["TARGET"] = targets["TARGET"].map(feature_dict[target_column]['to_int'])

## Dataframe creation and saving

In [38]:
ordinal_feature_names = traces[0].columns.difference(categorical_feature_names)
final_traces = copy.deepcopy(traces)

# Concatenate all features into one feature dataframe per trace
for i in range(0,len(traces)):
    # normalize values while encoding
    for cf in categorical_feature_names:
        final_traces[i][cf] = traces[i][cf].map(feature_dict[cf]['to_int'])
        #final_traces[i][cf] /= max(feature_dict[cf]['to_int'].values())
    
    ordinal_trace = traces[i][ordinal_feature_names]
    categorical_trace = traces[i][categorical_feature_names]
    final_traces[i] = pd.concat([ordinal_trace, categorical_trace, sp2_traces[i], pf_traces[i], targets], ignore_index=False, axis=1)

In [ ]:
assert(sum([len(t) for t in final_traces]) == len(eventlog_df))
pickle.dump(final_traces, open(traces_finalpath, "wb"), protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump(feature_dict, open(traces_dictionarypath, "wb"), protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
final_traces = pickle.load(open(traces_finalpath, "rb"))

In [12]:
import seaborn as sns
tlens = [len(t) for t in final_traces]
len_occurences = np.bincount(tlens)
ii = np.nonzero(len_occurences)[0]

for ln, nocc in zip(ii,len_occurences[ii]):
    print("{0:0>9d} occurs {1:0>10d}x".format(ln,nocc))

000000001 occurs 0000000004x
000000002 occurs 0000000059x
000000003 occurs 0000000049x
000000004 occurs 0000000046x
000000005 occurs 0000000043x
000000006 occurs 0000000032x
000000007 occurs 0000000034x
000000008 occurs 0000000021x
000000009 occurs 0000000022x
000000010 occurs 0000000015x
000000011 occurs 0000000016x
000000012 occurs 0000000009x
000000013 occurs 0000000009x
000000014 occurs 0000000011x
000000015 occurs 0000000009x
000000016 occurs 0000000002x
000000017 occurs 0000000003x
000000018 occurs 0000000005x
000000019 occurs 0000000008x
000000020 occurs 0000000005x
000000021 occurs 0000000004x
000000022 occurs 0000000003x
000000023 occurs 0000000003x
000000024 occurs 0000000004x
000000025 occurs 0000000002x
000000026 occurs 0000000006x
000000027 occurs 0000000006x
000000028 occurs 0000000007x
000000029 occurs 0000000007x
000000030 occurs 0000000012x
000000031 occurs 0000000009x
000000032 occurs 0000000011x
000000033 occurs 0000000005x
000000034 occurs 0000000007x
000000035 occu

In [10]:
len_occurences

array([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,   18,   19,   20,   21,   22,
         23,   24,   25,   26,   27,   28,   29,   30,   31,   32,   33,
         34,   35,   36,   37,   38,   39,   40,   41,   42,   43,   44,
         45,   46,   47,   48,   49,   50,   51,   52,   53,   54,   55,
         56,   57,   58,   59,   60,   62,   63,   64,   65,   66,   67,
         68,   69,   70,   71,   72,   73,   74,   75,   76,   77,   78,
         79,   80,   81,   82,   83,   84,   85,   86,   87,   88,   89,
         90,   91,   92,   93,   94,   95,   96,   97,   98,   99,  100,
        101,  102,  103,  104,  105,  106,  108,  109,  110,  111,  113,
        114,  115,  116,  117,  118,  119,  120,  121,  122,  123,  124,
        125,  126,  127,  128,  129,  130,  131,  132,  133,  134,  135,
        136,  137,  138,  139,  140,  141,  142,  144,  146,  147,  148,
        149,  150,  151,  152,  154,  155,  156,  1

In [ ]:
used_bytes = sum([t.memory_usage(index=True, deep=True).sum() for t in traces])
print("Memory usage of encoded data: {0} KB".format(used_bytes / 1024))
print("Memory usage of encoded data: {0} MB".format(used_bytes / 1024**2))
print("Memory usage of encoded data: {0} GB".format(used_bytes / 1024**3))

In [ ]:
trace_columns = traces[0].columns.tolist()
trace_columns = list(map(lambda e: bool(re.match('^TARGET$', e)), trace_columns))
trace_columns.index(True) # when do the target columns start?!